### **1. Эффективно ли комбинировать CatBoost, XGBoost и LightGBM, если они из одного семейства?**

CatBoost, XGBoost и LightGBM действительно относятся к одному семейству градиентного бустинга. Но это не означает, что их комбинация неэффективна. Они используют разные подходы в обучении (например, CatBoost хорошо работает с категориальными признаками, LightGBM оптимизирован по скорости, а XGBoost часто показывает лучшую точность на табличных данных). Комбинирование этих моделей может быть полезным, если они:
- Переобучаются на разных признаках.
- Имеют разную структуру ошибок.
- Слегка по-разному интерпретируют данные.

Однако в ансамблях действительно полезно комбинировать модели из **разных семейств**, например:
- **CatBoost** (градиентный бустинг),
- **SVM** (методы на основе опорных векторов),
- **LinearRegression** (линейные модели),
- **KNN** (алгоритмы на основе ближайших соседей).

Эти модели изучают данные с разных сторон и комбинируют их сильные стороны. 

---

### **2. Какой алгоритм лучше использовать в качестве мета-алгоритма?**

Мета-алгоритм (или модель второго уровня) обучается на предсказаниях базовых моделей. Хорошими вариантами могут быть:
- **Линейная регрессия** или **логистическая регрессия** (если данные достаточно простые и предсказания линейно зависимы).
- **CatBoost/XGBoost/LightGBM** для сложных задач с нелинейными зависимостями.
- **Neural Networks** для обработки сложных зависимостей, если данных достаточно много.

**Как выбирать?**
- Если данные небольшие — попробуй линейные модели (они быстрее).
- Если данных больше, стоит рассмотреть более мощные модели, такие как бустинг.

---

### **3. Чем отличаются Stacking и Blending?**

| **Метод**     | **Принцип работы**                                                                                          | **Преимущества**                                                        | **Недостатки**                                          |
|----------------|-----------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------|--------------------------------------------------------|
| **Stacking**   | Использует мета-алгоритм, обучающийся на предсказаниях базовых моделей, созданных на всей тренировочной выборке. | - Учитывает зависимости между предсказаниями моделей. <br>- Высокая точность. | - Более сложная реализация. <br>- Требует разделения данных на фолды. |
| **Blending**   | Использует простую линейную комбинацию или взвешенное усреднение предсказаний базовых моделей.              | - Простой в реализации. <br>- Не требует кросс-валидации.              | - Менее точный, так как игнорирует зависимости.         |

Blending проще, но менее точен. Если у тебя есть достаточные вычислительные мощности и время, **Stacking** чаще дает лучшие результаты.

---

### **4. Можно ли делать Stacking глубже?**

Да, можно углублять Stacking, создавая многоуровневые ансамбли. Например:
- На первом уровне: **CatBoost**, **SVM**, **LinearRegression**.
- На втором уровне: **CatBoost** как мета-алгоритм.
- Далее: результаты этого ансамбля можно стекать с другими моделями (например, **RandomForest**, **Neural Network**).

Это называется **глубоким стэкингом**. Но тут есть нюансы:
1. **Увеличение сложности:** Глубокие ансамбли сложно отлаживать и интерпретировать.
2. **Риск переобучения:** Если уровней слишком много, можно потерять обобщающую способность модели.
3. **Требовательность к ресурсам:** Углубленный стэкинг требует больших вычислительных мощностей.

---

### **Резюме**
1. **Можно комбинировать как модели одного семейства, так и разных классов.** Разные классы дают больше разнообразия, но даже модели одного семейства могут работать по-разному.
2. **Используй Stacking для точности, Blending — для простоты.**
3. **Углубленный Stacking возможен, но требует аккуратной настройки.** Убедись, что переобучение под контролем.
4. Экспериментируй с гиперпараметрами и составом ансамбля для поиска оптимального подхода.

### **5. Пример глубокого Stacking на Python**

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Генерация данных
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Первый уровень моделей
base_models = [
    ('catboost', CatBoostRegressor(verbose=0)),
    ('svm', SVR()),
    ('linear', LinearRegression())
]

# Первый уровень стэкинга
first_level = StackingRegressor(
    estimators=base_models,
    final_estimator=LinearRegression()
)
first_level.fit(X_train, y_train)

# Второй уровень: стекуем результат с RandomForest
final_model = StackingRegressor(
    estimators=[
        ('stacking', first_level),
        ('random_forest', RandomForestRegressor(random_state=42))
    ],
    final_estimator=CatBoostRegressor(verbose=0)
)
final_model.fit(X_train, y_train)

# Прогноз и оценка
y_pred = final_model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))

### **6. Расширенный стеккинг (stacked generalization with raw features), Out-of-Fold (OOF)**

Да, логика в вашем подходе есть. Добавление исходных признаков к метамодели в стекинге – вполне валидная и часто применяемая стратегия. Это называется Stacked Regression с использованием исходных признаков (Stacked Regressors with Original Features).

Почему это может быть полезно:

•  Метамодель получает больше контекста: Прогнозы моделей первого уровня (Catboost, XGBoost, KNN) могут быть сложными и не всегда легко интерпретируемыми. Добавление исходных признаков позволяет метамодели увидеть более прямую связь между характеристиками квартиры и ценой.
•  Исправление систематических ошибок: Если какая-то из базовых моделей систематически недооценивает или переоценивает квартиры определенного типа (например, квартиры в новостройках), метамодель с исходными признаками может это компенсировать.
•  Более точная калибровка: Метамодель может лучше откалибровать прогнозы базовых моделей, особенно если они имеют тенденцию быть слишком уверенными или слишком неуверенными в своих предсказаниях.

Как избежать утечки информации (data leakage):

Самая главная проблема в стекинге - это утечка данных, которая приводит к переобучению и плохой обобщающей способности на новых данных. Вот как её предотвратить:

1. Правильное разделение данных:
  •  Первый уровень (обучение базовых моделей): Обучите Catboost, XGBoost и KNN на тренировочном наборе данных. Важно, чтобы этот тренировочный набор был отделен от данных, используемых для обучения метамодели.
  •  Второй уровень (обучение метамодели):
    *  Сгенерируйте прогнозы Out-of-Fold (OOF) на тренировочном наборе: Вместо того чтобы просто предсказывать на тренировочном наборе, используйте кросс-валидацию (например, K-fold) на тренировочном наборе. Для каждого фолда:
      1. Обучите базовые модели (Catboost, XGBoost, KNN) на всех фолдах, кроме текущего.
      2. Предскажите стоимость на фолде, который был исключен из обучения.
    *  После завершения кросс-валидации у вас будет набор OOF прогнозов для каждой квартиры в тренировочном наборе. Эти OOF прогнозы используются в качестве признаков для метамодели.
    *  Важно: Исходные признаки квартир (площадь, этаж и т.д.) также должны быть присоединены к этим OOF прогнозам при обучении метамодели.
  •  Тестовый набор:
    1. Обучите базовые модели на всем тренировочном наборе.
    2. Предскажите стоимость квартир на тестовом наборе каждой базовой моделью.
    3. Объедините эти прогнозы с исходными признаками тестового набора.
    4. Используйте метамодель, обученную на OOF прогнозах и исходных признаках тренировочного набора, для предсказания стоимости на тестовом наборе.

2. Кросс-валидация для метамодели: Даже при использовании OOF прогнозов, полезно использовать кросс-валидацию при обучении метамодели. Это поможет оценить стабильность метамодели и избежать переобучения.

Прогнозы Out-of-Fold (OOF) — это прогнозы, полученные с использованием кросс-валидации, которые позволяют избежать утечки данных при обучении метамодели в стекинге. Давайте разберем это подробно:

Проблема утечки данных в стекинге (и почему нужны OOF прогнозы):

Представьте, что вы хотите построить модель, которая предсказывает вероятность того, что клиент покинет компанию (отток клиентов). Вы обучили несколько хороших моделей (например, логистическую регрессию, случайный лес, градиентный бустинг) и теперь хотите объединить их прогнозы, чтобы получить еще более точную модель.

•  Наивный подход (с утечкой данных): Вы берете весь свой тренировочный набор, обучаете на нем каждую из своих базовых моделей, а затем используете эти обученные модели для предсказания на том же тренировочном наборе. Затем вы берете эти прогнозы и используете их в качестве признаков для обучения метамодели (например, линейной регрессии). Это плохо!

Почему это плохо? Потому что каждая базовая модель "видела" данные, на которых она делала прогнозы. Это означает, что прогнозы базовых моделей будут переобучены на тренировочных данных. Метамодель, которая обучается на этих переобученных прогнозах, также будет переобучена и, скорее всего, покажет плохие результаты на новых, невиданных данных. Это называется утечкой данных (data leakage).

Out-of-Fold (OOF) прогнозы: Решение проблемы утечки данных

OOF прогнозы — это способ создания прогнозов для тренировочного набора, при котором каждая базовая модель делает прогноз для конкретной строки тренировочного набора, не видя эту строку во время обучения. Это достигается с помощью кросс-валидации.

Как это работает (пошагово):

1. Разделите тренировочный набор на K частей (фолдов). Например, K=5.
2. Для каждого фолда (от 1 до K):
  •  Обучите базовые модели (логистическую регрессию, случайный лес, градиентный бустинг и т. д.) на всех фолдах, кроме текущего. Например, если текущий фолд — 1, вы обучаете модели на фолдах 2, 3, 4 и 5.
  •  Сделайте прогнозы только на текущем фолде. Используйте обученные модели для предсказания целевой переменной для строк в фолде 1.
3. Повторите шаг 2 для каждого фолда. В конце этого процесса каждая строка в тренировочном наборе получит прогноз от каждой базовой модели. Но важно то, что этот прогноз был сделан моделью, которая не обучалась на этой конкретной строке.
4. Соберите все прогнозы: У вас будет набор прогнозов OOF для каждой базовой модели. Эти OOF прогнозы теперь можно безопасно использовать в качестве признаков для обучения метамодели.

Почему OOF прогнозы предотвращают утечку данных:

Потому что каждая базовая модель делает прогноз для каждой строки в тренировочном наборе, не видя эту строку во время обучения. Это имитирует ситуацию, когда модель делает прогноз на новых, невиданных данных. Это гарантирует, что метамодель не будет обучаться на переобученных прогнозах и будет лучше обобщать на новые данные.

In [49]:
import pandas as pd
import numpy as np

# Установите seed для воспроизводимости результатов
np.random.seed(42)

# Количество квартир
n_samples = 1000

# Генерация признаков
data = pd.DataFrame({
    'area': np.random.randint(30, 120, n_samples),  # Площадь (кв.м)
    'floor': np.random.randint(1, 25, n_samples),   # Этаж
    'building_age': np.random.randint(1, 50, n_samples), # Возраст дома (лет)
    'distance_to_metro': np.random.randint(500, 5000, n_samples), # Расстояние до метро (м)
    'latitude': np.random.uniform(55.6, 55.9, n_samples), # Широта (Москва, пример)
    'longitude': np.random.uniform(37.4, 37.8, n_samples),# Долгота (Москва, пример)
    'condition': np.random.choice(['good', 'average', 'bad'], n_samples),  # Состояние квартиры
    'building_type': np.random.choice(['brick', 'panel', 'monolith'], n_samples), # Тип дома
    'rooms': np.random.randint(1, 4, n_samples)  # Количество комнат
})

# Закодируем категориальные признаки (condition и building_type)
data = pd.get_dummies(data, columns=['condition', 'building_type'], drop_first=True)

# Создадим целевую переменную (цена) на основе признаков с добавлением шума
# Это пример, коэффициенты можно менять, чтобы модель была сложнее
price = (
    100000 * data['area'] +
    1000 * data['floor'] -
    500 * data['building_age'] -
    20 * data['distance_to_metro'] +
    100000 * data['rooms'] +
    50000 * data['condition_good'] +
    80000 * data['building_type_monolith'] +
    60000 * data['building_type_panel'] +
    np.random.normal(0, 500000, n_samples)  # Добавляем случайный шум
)

# Обеспечим, чтобы цена была положительной
price = np.maximum(price, 1000000) #Минимальная цена квартиры 1 млн

data['price'] = price

# Разделим на X и y
X = data.drop('price', axis=1)
y = data['price']

In [50]:
X

,area,floor,building_age,distance_to_metro,latitude,longitude,rooms,condition_bad,condition_good,building_type_monolith,building_type_panel
0,81,11,12,2590,55.738515,37.454184,1,False,False,False,True
1,44,9,1,4169,55.876298,37.492590,2,False,True,False,False
2,101,5,20,4772,55.808379,37.747364,1,True,False,True,False
3,90,14,28,2133,55.818694,37.770586,3,False,False,False,True
4,50,3,43,2017,55.858507,37.567851,2,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
995,93,1,33,1604,55.648906,37.722039,2,False,True,False,False
996,51,5,17,3036,55.670144,37.738016,2,True,False,False,False
997,89,20,24,4699,55.607112,37.558021,2,False,True,False,True
998,93,8,34,4354,55.850484,37.420505,2,False,True,False,True


In [51]:
from sklearn.model_selection import train_test_split

# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import catboost as cb
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd

# Определите базовые модели
catboost = cb.CatBoostRegressor(verbose=False)
xgboost = xgb.XGBRegressor()
knn = KNeighborsRegressor()

# Определите метамодель
meta_model = LinearRegression()

# Определите количество фолдов для кросс-валидации
n_folds = 5

# Создайте KFold объект
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Создайте массивы для хранения OOF прогнозов
oof_catboost = np.zeros(len(X_train))
oof_xgboost = np.zeros(len(X_train))
oof_knn = np.zeros(len(X_train))

val_idx = []

# Цикл по фолдам
for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"Fold {fold+1}")
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Обучите базовые модели на тренировочном фолде
    catboost.fit(X_tr, y_tr)
    xgboost.fit(X_tr, y_tr)
    knn.fit(X_tr, y_tr)

    # Сделайте прогнозы на валидационном фолде
    oof_catboost[val_index] = catboost.predict(X_val)
    oof_xgboost[val_index] = xgboost.predict(X_val)
    oof_knn[val_index] = knn.predict(X_val)

    val_idx.append(val_index)

# Создайте DataFrame с OOF прогнозами и исходными признаками
meta_features = pd.DataFrame({
                              'catboost': oof_catboost,
                              'xgboost': oof_xgboost,
                              'knn': oof_knn
                            })

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [53]:
meta_features

,catboost,xgboost,knn
0,7.111898e+06,6850361.50,7.050429e+06
1,4.418849e+06,4628059.50,6.516811e+06
2,5.536954e+06,5179128.50,5.641945e+06
3,1.126680e+07,11524882.00,1.063361e+07
4,1.055839e+07,10574124.00,8.672891e+06
...,...,...,...
795,1.159889e+07,11701244.00,8.199318e+06
796,7.504467e+06,7520556.00,9.073351e+06
797,3.499754e+06,3640001.75,5.807577e+06
798,7.669577e+06,7822871.00,7.664489e+06


In [54]:
val_idx

[array([  2,   7,  10,  23,  29,  30,  31,  33,  39,  49,  54,  63,  65,
         66,  67,  72,  76,  77,  78,  81,  84,  86,  96,  97, 101, 109,
        110, 118, 120, 137, 139, 155, 168, 174, 192, 198, 199, 204, 208,
        209, 210, 211, 215, 218, 231, 235, 244, 250, 254, 260, 265, 266,
        275, 281, 286, 294, 296, 302, 306, 314, 316, 323, 326, 327, 333,
        336, 346, 352, 357, 360, 361, 365, 367, 368, 377, 383, 388, 393,
        395, 398, 409, 422, 423, 425, 428, 432, 446, 456, 464, 481, 483,
        486, 490, 506, 512, 513, 515, 519, 521, 525, 526, 529, 532, 533,
        534, 537, 545, 568, 570, 578, 589, 591, 594, 595, 596, 604, 608,
        610, 621, 622, 628, 635, 637, 640, 641, 644, 652, 655, 656, 658,
        662, 666, 667, 670, 688, 692, 696, 705, 715, 719, 720, 721, 723,
        738, 741, 744, 746, 750, 754, 758, 760, 764, 776, 777, 783, 786,
        787, 795, 796, 798]),
 array([  6,  11,  18,  24,  28,  41,  42,  43,  44,  51,  55,  56,  60,
         61,  69,  70

In [55]:
# Объедините OOF прогнозы с исходными признаками
X_train_ = X_train
X_train_.reset_index(drop = True, inplace = True)
meta_features = pd.concat([meta_features, X_train_], axis=1)

# Обучите метамодель на OOF прогнозах и исходных признаках
meta_model.fit(meta_features, y_train)

# 1. Обучите базовые модели на всем тренировочном наборе
catboost.fit(X_train, y_train)
xgboost.fit(X_train, y_train)
knn.fit(X_train, y_train)

# 2. Сделайте прогнозы на тестовом наборе
test_catboost = catboost.predict(X_test)  # X_test - ваш тестовый набор
test_xgboost = xgboost.predict(X_test)
test_knn = knn.predict(X_test)

# 3. Создайте DataFrame с прогнозами и исходными признаками тестового набора
test_meta_features = pd.DataFrame({
                                   'catboost': test_catboost,
                                   'xgboost': test_xgboost,
                                   'knn': test_knn
                                 })

X_test_ = X_test
X_test_.reset_index(drop = True, inplace = True)
test_meta_features = pd.concat([test_meta_features, X_test_], axis=1)

# 4. Сделайте предсказания на тестовом наборе с помощью метамодели
predictions = meta_model.predict(test_meta_features)

In [56]:
predictions

array([ 6554218.87391458,  7601791.59477856,  9290475.15469377,
       11735813.74102364,  3556886.8376087 ,  7048998.60414339,
        3308298.83303774,  3779860.47576784, 11925078.58988766,
        3547577.28422388,  5755742.61586154,  6349691.18537873,
        7497037.67829323,  9288221.13878348,  8186938.91866433,
        5859456.71270305,  6706741.77346121,  4319593.27192665,
        4603559.94795701,  6046854.6611786 ,  9318266.18994587,
       11757299.65772441,  4219259.02516267,  6665345.70292857,
        5014454.70806822,  5031590.24952122, 10186479.00524811,
        8180120.94247629,  8799360.03283724,  5171220.99113298,
        5725659.9145285 ,  5738251.37570492,  5115437.24400549,
        6046617.96478718,  5469130.26138715,  4381694.97759678,
        5843045.7869966 ,  3666598.69530463,  8941889.49133173,
        3701686.60246806,  7848179.84642658,  6060887.12059211,
        8181255.44193326, 10416327.26614696,  5161881.00744631,
        5631460.85865947,  9688075.10040

In [58]:
y_test

521    6.637282e+06
737    8.440091e+06
740    9.027947e+06
660    1.237589e+07
411    3.748069e+06
           ...     
408    9.677073e+06
332    7.761302e+06
208    1.117080e+07
613    1.115507e+07
78     9.301880e+06
Name: price, Length: 200, dtype: float64

#### **Deep seek**

In [10]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

# Создаем искусственные данные
np.random.seed(42)
n_samples = 1000

# Генерируем случайные значения для каждого фактора
area = np.random.uniform(30, 150, n_samples)  # Площадь квартиры
floor = np.random.randint(1, 20, n_samples)   # Этаж
total_floors = np.random.randint(floor, floor + 10, n_samples)  # Этажность здания
year_built = np.random.randint(1950, 2020, n_samples)  # Год постройки
condition = np.random.choice(['good', 'average', 'bad'], n_samples)  # Состояние отделки

# Приведение состояния к числовым значениям
data = pd.DataFrame({
    'area': area,
    'floor': floor,
    'total_floors': total_floors,
    'year_built': year_built,
    'condition': condition
})

# Приведение состояния к числовым значениям
data['condition_numeric'] = data['condition'].map({'good': 3, 'average': 2, 'bad': 1})

# Генерируем целевую переменную (цена квартиры) на основе факторов
price = (
    data['area'] * 1000 +
    data['floor'] * 500 +
    data['total_floors'] * 100 +
    (2020 - data['year_built']) * 300 +
    data['condition_numeric'] * 1000
)
data['price'] = price

# Добавляем информацию о районе
#districts = np.random.choice(['District1', 'District2', 'District3'], n_samples)  # Пример 3 района
#data['district'] = districts

# Добавляем координаты и расстояние до метро (искусственные данные)
latitude = np.random.uniform(55.5, 56.0, n_samples)  # Пример координат широты
longitude = np.random.uniform(37.0, 38.0, n_samples)  # Пример координат долготы
distance_to_metro = np.random.uniform(1000, 5000, n_samples)  # Пример расстояния до метро

data['latitude'] = latitude
data['longitude'] = longitude
data['distance_to_metro'] = distance_to_metro

# Разделение на признаки и целевую переменную
X = data.drop(['price', 'condition'], axis=1)
y = data['price']

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем KFold для кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Создаем массивы для мета-признаков
catboost_meta_train = np.zeros((X_train.shape[0],))
xgboost_meta_train = np.zeros((X_train.shape[0],))
knn_meta_train = np.zeros((X_train.shape[0],))

catboost_meta_test = np.zeros((X_test.shape[0],))
xgboost_meta_test = np.zeros((X_test.shape[0],))
knn_meta_test = np.zeros((X_test.shape[0],))

# Обучаем основные модели и создаем мета-признаки
for fold, (train_index, valid_index) in enumerate(kf.split(X_train)):
    X_fold_train, X_fold_valid = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_fold_train, y_fold_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    # Обучаем CatBoost
    catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='RMSE', verbose=0)
    catboost_model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_valid, y_fold_valid), early_stopping_rounds=10, use_best_model=True)
    catboost_predictions = catboost_model.predict(X_fold_valid)
    catboost_meta_train[valid_index] = catboost_predictions
    
    # Обучаем XGBoost
    xgboost_model = XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=6, objective='reg:squarederror', random_state=42)
    xgboost_model.fit(X_fold_train, y_fold_train)
    xgboost_predictions = xgboost_model.predict(X_fold_valid)
    xgboost_meta_train[valid_index] = xgboost_predictions
    
    # Обучаем KNN
    knn_model = KNeighborsRegressor(n_neighbors=5)
    knn_model.fit(X_fold_train, y_fold_train)
    knn_predictions = knn_model.predict(X_fold_valid)
    knn_meta_train[valid_index] = knn_predictions
    
    # Обучаем CatBoost на тестовых данных для финальных предсказаний
    catboost_meta_test += catboost_model.predict(X_test) / kf.n_splits # усреднение, получаем прогноз и делим его на количество фолдов = 5
    
    # Обучаем XGBoost на тестовых данных для финальных предсказаний
    xgboost_meta_test += xgboost_model.predict(X_test) / kf.n_splits  # усреднение
    
    # Обучаем KNN на тестовых данных для финальных предсказаний
    knn_meta_test += knn_model.predict(X_test) / kf.n_splits          # усреднение

# Создаем DataFrame с мета-признаками и оригинальными признаками для обучения метамодели
meta_X_train = pd.DataFrame({
                             'catboost': catboost_meta_train,
                             'xgboost': xgboost_meta_train,
                             'knn': knn_meta_train
                           })
meta_X_train = pd.concat([meta_X_train, X_train.reset_index(drop=True)], axis=1)

# Создаем DataFrame с мета-признаками и оригинальными признаками для предсказания метамоделью
meta_X_test = pd.DataFrame({
                            'catboost': catboost_meta_test,
                            'xgboost': xgboost_meta_test,
                            'knn': knn_meta_test
                        })
meta_X_test = pd.concat([meta_X_test, X_test.reset_index(drop=True)], axis=1)

# Обучаем метамодель (например, Ridge)
meta_model = Ridge()
meta_model.fit(meta_X_train, y_train)

# Предсказание на тестовых данных
final_predictions = meta_model.predict(meta_X_test)

# Оцениваем модель
mse = mean_squared_error(y_test, final_predictions)
print(f"Mean Squared Error: {mse:.2f}")

Mean Squared Error: 5.42


In [14]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Создаем искусственные данные
np.random.seed(42)
n_samples = 1000

# Генерируем случайные значения для каждого фактора
area = np.random.uniform(30, 150, n_samples)  # Площадь квартиры
floor = np.random.randint(1, 20, n_samples)   # Этаж
total_floors = np.random.randint(floor, floor + 10, n_samples)  # Этажность здания
year_built = np.random.randint(1950, 2020, n_samples)  # Год постройки
condition = np.random.choice(['good', 'average', 'bad'], n_samples)  # Состояние отделки

# Приведение состояния к числовым значениям
data = pd.DataFrame({
    'area': area,
    'floor': floor,
    'total_floors': total_floors,
    'year_built': year_built,
    'condition': condition
})

# Приведение состояния к числовым значениям
data['condition_numeric'] = data['condition'].map({'good': 3, 'average': 2, 'bad': 1})

# Генерируем целевую переменную (цена квартиры) на основе факторов
price = (
    data['area'] * 1000 +
    data['floor'] * 500 +
    data['total_floors'] * 100 +
    (2020 - data['year_built']) * 300 +
    data['condition_numeric'] * 1000
)
data['price'] = price

# Добавляем информацию о районе
districts = np.random.choice(['District1', 'District2', 'District3'], n_samples)  # Пример 3 района
data['district'] = districts

# Добавляем координаты и расстояние до метро (искусственные данные)
latitude = np.random.uniform(55.5, 56.0, n_samples)  # Пример координат широты
longitude = np.random.uniform(37.0, 38.0, n_samples)  # Пример координат долготы
distance_to_metro = np.random.uniform(1000, 5000, n_samples)  # Пример расстояния до метро

data['latitude'] = latitude
data['longitude'] = longitude
data['distance_to_metro'] = distance_to_metro

# Разделение на признаки и целевую переменную
X = data.drop(['price', 'condition'], axis=1)
y = data['price']

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обработка категориальных признаков
encoder = OneHotEncoder()
district_encoded = encoder.fit_transform(X_train[['district']])
district_encoded_test = encoder.transform(X_test[['district']])

# Добавляем закодированные признаки в DataFrame
X_train_encoded = pd.concat([X_train.drop('district', axis=1).reset_index(drop=True), pd.DataFrame(district_encoded)], axis=1)
X_test_encoded = pd.concat([X_test.drop('district', axis=1).reset_index(drop=True), pd.DataFrame(district_encoded_test)], axis=1)

# Создаем KFold для кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Создаем массивы для мета-признаков
catboost_meta_train = np.zeros((X_train_encoded.shape[0],))
xgboost_meta_train = np.zeros((X_train_encoded.shape[0],))
knn_meta_train = np.zeros((X_train_encoded.shape[0],))

catboost_meta_test = np.zeros((X_test_encoded.shape[0],))
xgboost_meta_test = np.zeros((X_test_encoded.shape[0],))
knn_meta_test = np.zeros((X_test_encoded.shape[0],))

# Обучаем основные модели и создаем мета-признаки
for fold, (train_index, valid_index) in enumerate(kf.split(X_train_encoded)):
    X_fold_train, X_fold_valid = X_train_encoded.iloc[train_index], X_train_encoded.iloc[valid_index]
    y_fold_train, y_fold_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    # Обучаем CatBoost
    catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='RMSE', verbose=0)
    catboost_model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_valid, y_fold_valid), early_stopping_rounds=10, use_best_model=True)
    catboost_predictions = catboost_model.predict(X_fold_valid)
    catboost_meta_train[valid_index] = catboost_predictions
    
    # Обучаем XGBoost
    xgboost_model = XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=6, objective='reg:squarederror', random_state=42)
    xgboost_model.fit(X_fold_train, y_fold_train)
    xgboost_predictions = xgboost_model.predict(X_fold_valid)
    xgboost_meta_train[valid_index] = xgboost_predictions
    
    # Обучаем KNN
    knn_model = KNeighborsRegressor(n_neighbors=5)
    knn_model.fit(X_fold_train, y_fold_train)
    knn_predictions = knn_model.predict(X_fold_valid)
    knn_meta_train[valid_index] = knn_predictions
    
    # Обучаем CatBoost на тестовых данных для финальных предсказаний
    catboost_meta_test += catboost_model.predict(X_test_encoded) / kf.n_splits
    
    # Обучаем XGBoost на тестовых данных для финальных предсказаний
    xgboost_meta_test += xgboost_model.predict(X_test_encoded) / kf.n_splits
    
    # Обучаем KNN на тестовых данных для финальных предсказаний
    knn_meta_test += knn_model.predict(X_test_encoded) / kf.n_splits

# Создаем DataFrame с мета-признаками и оригинальными признаками для обучения метамодели
meta_X_train = pd.DataFrame({
    'catboost': catboost_meta_train,
    'xgboost': xgboost_meta_train,
    'knn': knn_meta_train
})
meta_X_train = pd.concat([meta_X_train, X_train_encoded.reset_index(drop=True)], axis=1)

# Создаем DataFrame с мета-признаками и оригинальными признаками для предсказания метамоделью
meta_X_test = pd.DataFrame({
    'catboost': catboost_meta_test,
    'xgboost': xgboost_meta_test,
    'knn': knn_meta_test
})
meta_X_test = pd.concat([meta_X_test, X_test_encoded.reset_index(drop=True)], axis=1)

# Обучаем метамодель (например, Ridge)
meta_model = Ridge()
meta_model.fit(meta_X_train, y_train)

# Предсказание на тестовых данных
final_predictions = meta_model.predict(meta_X_test)

# Оцениваем модель
mse = mean_squared_error(y_test, final_predictions)
print(f"Mean Squared Error: {mse:.2f}")

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=8]="  (0, 0)	1.0": Cannot convert obj   (0, 0)	1.0 to float

In [ ]:
from sklearn.model_selection import GridSearchCV

# Определяем параметры для подбора
param_grid = {
    'iterations': [1000],
    'learning_rate': [0.1, 0.05],
    'depth': [6, 8],
    'l2_leaf_reg': [3, 5],
    'early_stopping_rounds': [10]
}

# Создаем модель CatBoost
catboost_model = CatBoostRegressor(loss_function='RMSE', verbose=0)

# Используем GridSearchCV для подбора гиперпараметров
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

# Лучшие параметры
best_params = grid_search.best_params_
print(f"Лучшие параметры для CatBoost: {best_params}")

# Обучаем лучшую модель CatBoost
best_catboost_model = CatBoostRegressor(**best_params, loss_function='RMSE', verbose=0)
best_catboost_model.fit(X_train_encoded, y_train, eval_set=(X_test_encoded, y_test), early_stopping_rounds=10, use_best_model=True)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Определяем параметры для подбора
param_grid = {
    'n_estimators': [1000],
    'learning_rate': [0.1, 0.05],
    'max_depth': [6, 8],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Создаем модель XGBoost
xgboost_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Используем GridSearchCV для подбора гиперпараметров
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

# Лучшие параметры
best_params = grid_search.best_params_
print(f"Лучшие параметры для XGBoost: {best_params}")

# Обучаем лучшую модель XGBoost
best_xgboost_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
best_xgboost_model.fit(X_train_encoded, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Определяем параметры для подбора
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Создаем модель KNN
knn_model = KNeighborsRegressor()

# Используем GridSearchCV для подбора гиперпараметров
grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

# Лучшие параметры
best_params = grid_search.best_params_
print(f"Лучшие параметры для KNN: {best_params}")

# Обучаем лучшую модель KNN
best_knn_model = KNeighborsRegressor(**best_params)
best_knn_model.fit(X_train_encoded, y_train)